<a href="https://colab.research.google.com/github/michaelprinc/PySpark/blob/master/MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge

## Download and install Spark

In [0]:
!ls

sample_data


In [0]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [74.7 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [786 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

## Setup environment

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Downloading and preprocessing data

In [31]:
link = 'https://drive.google.com/open?id=1C5ll7Iv_WGEphfnWpXv0idcZ9HkmZcrg'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('employee.txt')  
#df3 = pd.read_csv('Filename.csv')
# Dataset is now stored in a Pandas Dataframe
!ls -l

1C5ll7Iv_WGEphfnWpXv0idcZ9HkmZcrg
total 220772
-rw-r--r--  1 root root      2620 Nov 17 19:28  adc.json
-rw-r--r--  1 root root     96027 Nov 17 19:33  employee.txt
-rw-r--r--  1 root root     68396 Nov 17 19:21 'open?id=11PxbOlyNEwihWUoIqascgLaeMtW1isie'
drwxr-xr-x  1 root root      4096 Nov  6 16:17  sample_data
drwxrwxr-x 13 1000 1000      4096 Jun  1  2018  spark-2.3.1-bin-hadoop2.7
-rw-r--r--  1 root root 225883783 Jun  1  2018  spark-2.3.1-bin-hadoop2.7.tgz
drwxr-xr-x  2 root root      4096 Nov 17 19:13  spark-warehouse


In [0]:
#!mv open?id=11PxbOlyNEwihWUoIqascgLaeMtW1isie employee.txt
#!ls -l

mv: cannot stat 'open?id=11PxbOlyNEwihWUoIqascgLaeMtW1isie': No such file or directory
total 220672
-rw-r--r--  1 root root     68393 Nov 17 19:10 employee.txt
drwxr-xr-x  1 root root      4096 Nov  6 16:17 sample_data
drwxrwxr-x 13 1000 1000      4096 Jun  1  2018 spark-2.3.1-bin-hadoop2.7
-rw-r--r--  1 root root 225883783 Jun  1  2018 spark-2.3.1-bin-hadoop2.7.tgz
drwxr-xr-x  2 root root      4096 Nov 17 19:13 spark-warehouse


In [32]:
from pyspark.sql.functions import to_timestamp,col,lit
emp_df = spark.read.csv('employee.txt',header=True)
emp_df

DataFrame[id: string, last_name: string, email: string, gender: string, department: string, start_date: string, salary: string, job_title: string, region_id: string]

## Challenge questions

**MLlib**

In [33]:
emp_df.schema

StructType(List(StructField(id,StringType,true),StructField(last_name,StringType,true),StructField(email,StringType,true),StructField(gender,StringType,true),StructField(department,StringType,true),StructField(start_date,StringType,true),StructField(salary,StringType,true),StructField(job_title,StringType,true),StructField(region_id,StringType,true)))

In [34]:
emp_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- department: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- region_id: string (nullable = true)



In [35]:
emp_df.columns
emp_df.take(5)

[Row(id='1', last_name="'Kelley'", email="'rkelley0@soundcloud.com'", gender="'Female'", department="'Computers'", start_date="'10/2/2009'", salary='67470', job_title="'Structural Engineer'", region_id='2'),
 Row(id='2', last_name="'Armstrong'", email="'sarmstrong1@infoseek.co.jp'", gender="'Male'", department="'Sports'", start_date="'3/31/2008'", salary='71869', job_title="'Financial Advisor'", region_id='2'),
 Row(id='3', last_name="'Carr'", email="'fcarr2@woothemes.com'", gender="'Male'", department="'Automotive'", start_date="'7/12/2009'", salary='101768', job_title="'Recruiting Manager'", region_id='3'),
 Row(id='4', last_name="'Murray'", email="'jmurray3@gov.uk'", gender="'Female'", department="'Jewelery'", start_date="'12/25/2014'", salary='96897', job_title="'Desktop Support Technician'", region_id='3'),
 Row(id='5', last_name="'Ellis'", email="'jellis4@sciencedirect.com'", gender="'Female'", department="'Grocery'", start_date="'9/19/2002'", salary='63702', job_title="'Software

In [36]:
emp_df.count()

1000

Sampling / Managers Data Frame

In [37]:
sample_df = emp_df.sample(False,0.1)
sample_df.count()

92

In [39]:
emp_mgrs_df = emp_df.filter("salary >= 100000")
emp_mgrs_df.count()

478

In [40]:
emp_mgrs_df.select("salary").show(20)

+------+
|salary|
+------+
|101768|
|118497|
|108657|
|108093|
|121966|
|141139|
|106659|
|148952|
|109890|
|115274|
|144724|
|126103|
|144965|
|113507|
|120579|
|107222|
|125668|
|113857|
|108378|
|133424|
+------+
only showing top 20 rows

